# 🎬 AI Video Generator - Optimized for Google Colab

**Features:**
- 720p Resolution Support
- 10-second video generation
- 24fps output
- Realistic human posing
- Batch processing
- Memory optimization
- Progress tracking
- Automatic environment detection
- **FIXED:** GPUtil dependency issues resolved

**Requirements:**
- GPU Runtime (T4 or better recommended)
- ~15GB RAM
- ~20GB Disk Space

---

## 📋 Step 1: Environment Detection & Setup (FIXED)

In [ ]:
import os
import sys
import subprocess
import platform
import psutil
from pathlib import Path
import time
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

# Fixed GPUtil import with fallback
try:
    import GPUtil
    GPUTIL_AVAILABLE = True
    print("✅ GPUtil imported successfully")
except ImportError:
    GPUTIL_AVAILABLE = False
    print("⚠️ GPUtil not available, using alternative GPU detection")

class EnvironmentDetector:
    def __init__(self):
        self.system_info = {}
        self.requirements_met = False
        
    def detect_environment(self):
        """Detect and validate the current environment"""
        print("🔍 Detecting environment...")
        
        # Basic system info
        self.system_info['platform'] = platform.system()
        self.system_info['python_version'] = sys.version
        self.system_info['is_colab'] = 'google.colab' in sys.modules
        
        # Memory info
        memory = psutil.virtual_memory()
        self.system_info['total_ram_gb'] = round(memory.total / (1024**3), 2)
        self.system_info['available_ram_gb'] = round(memory.available / (1024**3), 2)
        
        # GPU info with fallback
        self._detect_gpu_info()
        
        # Disk space
        disk_usage = psutil.disk_usage('/')
        self.system_info['disk_free_gb'] = round(disk_usage.free / (1024**3), 2)
        
        self._display_info()
        self._check_requirements()
        
    def _detect_gpu_info(self):
        """Detect GPU information with multiple fallback methods"""
        # Method 1: Try PyTorch first (most reliable)
        try:
            import torch
            self.system_info['cuda_available'] = torch.cuda.is_available()
            if torch.cuda.is_available():
                self.system_info['gpu_name'] = torch.cuda.get_device_name(0)
                self.system_info['gpu_memory_gb'] = round(torch.cuda.get_device_properties(0).total_memory / (1024**3), 2)
                print("✅ GPU detected via PyTorch")
                return
        except ImportError:
            pass
        except Exception as e:
            print(f"⚠️ PyTorch GPU detection failed: {str(e)}")
            
        # Method 2: Try GPUtil if available
        if GPUTIL_AVAILABLE:
            try:
                gpus = GPUtil.getGPUs()
                if gpus:
                    self.system_info['cuda_available'] = True
                    self.system_info['gpu_name'] = gpus[0].name
                    self.system_info['gpu_memory_gb'] = round(gpus[0].memoryTotal / 1024, 2)
                    print("✅ GPU detected via GPUtil")
                    return
            except Exception as e:
                print(f"⚠️ GPUtil detection failed: {str(e)}")
        
        # Method 3: Try nvidia-smi command
        try:
            result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader,nounits'], 
                                  capture_output=True, text=True, timeout=5)
            if result.returncode == 0 and result.stdout.strip():
                lines = result.stdout.strip().split('\n')
                if lines:
                    parts = lines[0].split(', ')
                    if len(parts) >= 2:
                        self.system_info['cuda_available'] = True
                        self.system_info['gpu_name'] = parts[0].strip()
                        self.system_info['gpu_memory_gb'] = round(float(parts[1]) / 1024, 2)
                        print("✅ GPU detected via nvidia-smi")
                        return
        except (subprocess.TimeoutExpired, FileNotFoundError, ValueError):
            pass
        except Exception as e:
            print(f"⚠️ nvidia-smi detection failed: {str(e)}")
            
        # No GPU detected
        self.system_info['cuda_available'] = False
        self.system_info['gpu_name'] = 'None'
        self.system_info['gpu_memory_gb'] = 0
        print("ℹ️ No GPU detected - running in CPU mode")
        
    def _display_info(self):
        """Display system information in a nice format"""
        info_html = f"""
        <div style="background-color: #f0f2f6; padding: 15px; border-radius: 10px; margin: 10px 0;">
            <h3>🖥️ System Information</h3>
            <table style="width: 100%; border-collapse: collapse;">
                <tr><td><b>Platform:</b></td><td>{self.system_info['platform']}</td></tr>
                <tr><td><b>Google Colab:</b></td><td>{'✅ Yes' if self.system_info['is_colab'] else '❌ No'}</td></tr>
                <tr><td><b>Total RAM:</b></td><td>{self.system_info['total_ram_gb']} GB</td></tr>
                <tr><td><b>Available RAM:</b></td><td>{self.system_info['available_ram_gb']} GB</td></tr>
                <tr><td><b>GPU:</b></td><td>{self.system_info['gpu_name']}</td></tr>
                <tr><td><b>GPU Memory:</b></td><td>{self.system_info['gpu_memory_gb']} GB</td></tr>
                <tr><td><b>Free Disk Space:</b></td><td>{self.system_info['disk_free_gb']} GB</td></tr>
            </table>
        </div>
        """
        display(HTML(info_html))
        
    def _check_requirements(self):
        """Check if system meets requirements"""
        issues = []
        
        if not self.system_info['cuda_available']:
            issues.append("⚠️ CUDA not available - will use CPU (slower performance)")
        elif self.system_info['gpu_memory_gb'] < 6:
            issues.append("⚠️ GPU memory < 6GB - may experience memory issues")
            
        if self.system_info['total_ram_gb'] < 12:
            issues.append("⚠️ RAM < 12GB - may experience memory issues")
            
        if self.system_info['disk_free_gb'] < 15:
            issues.append("⚠️ Disk space < 15GB - may not have enough space for models")
            
        if issues:
            print("\n⚠️ System Issues Detected:")
            for issue in issues:
                print(f"  {issue}")
            print("\n💡 Recommendation: Use Google Colab Pro with GPU runtime for best results")
        else:
            print("\n✅ All requirements met! Ready to proceed.")
            self.requirements_met = True
            
        return len(issues) == 0

# Initialize and run environment detection
env_detector = EnvironmentDetector()
env_detector.detect_environment()

## 📦 Step 2: Install Dependencies (UPDATED)

In [ ]:
class DependencyManager:
    def __init__(self):
        self.packages = {
            'essential': [
                'torch>=2.0.0',
                'torchvision>=0.15.0',
                'diffusers>=0.34.0',
                'transformers>=4.53.0',
                'accelerate>=1.8.0',
                'xformers>=0.0.22',
            ],
            'video': [
                'opencv-python-headless>=4.11.0',
                'imageio[ffmpeg]>=2.37.0',
                'moviepy>=2.2.0',
                'av>=10.0.0',
                'decord>=0.6.0'
            ],
            'ui': [
                'streamlit>=1.28.0',
                'gradio>=4.0.0',
                'ipywidgets>=8.1.0',
                'matplotlib>=3.10.0',
                'pillow>=10.0.0'
            ],
            'utils': [
                'huggingface-hub>=0.33.0',
                'safetensors>=0.5.0',
                'einops>=0.7.0',
                'omegaconf>=2.3.0',
                'pyngrok>=7.0.0',
                'GPUtil>=1.4.0'  # Added GPUtil to dependencies
            ]
        }
        
    def install_packages(self, category='all'):
        """Install packages with progress tracking"""
        categories = [category] if category != 'all' else self.packages.keys()
        
        for cat in categories:
            packages = self.packages[cat]
            print(f"\n📦 Installing {cat} packages...")
            
            progress_bar = tqdm(packages, desc=f"Installing {cat}")
            for package in progress_bar:
                try:
                    progress_bar.set_postfix_str(f"Installing {package.split('>=')[0]}")
                    result = subprocess.run(
                        [sys.executable, '-m', 'pip', 'install', package, '--quiet'],
                        capture_output=True,
                        text=True,
                        timeout=300
                    )
                    if result.returncode != 0:
                        print(f"\n⚠️ Warning: Failed to install {package}")
                        print(f"Error: {result.stderr}")
                except subprocess.TimeoutExpired:
                    print(f"\n⚠️ Timeout installing {package}")
                except Exception as e:
                    print(f"\n⚠️ Error installing {package}: {str(e)}")
                    
        print("\n✅ Dependencies installation completed!")
        
    def verify_installation(self):
        """Verify critical packages are installed"""
        critical_imports = {
            'torch': 'PyTorch',
            'diffusers': 'Diffusers',
            'transformers': 'Transformers',
            'cv2': 'OpenCV',
            'imageio': 'ImageIO',
            'streamlit': 'Streamlit'
        }
        
        # Also verify GPUtil with fallback
        print("\n🔍 Verifying installations...")
        
        # Check GPUtil specifically
        try:
            import GPUtil
            print("✅ GPUtil - OK")
        except ImportError:
            print("⚠️ GPUtil - NOT FOUND (will use alternatives)")
            
        for module, name in critical_imports.items():
            try:
                __import__(module)
                print(f"✅ {name} - OK")
            except ImportError:
                print(f"❌ {name} - FAILED")
                
        # Check CUDA availability
        try:
            import torch
            if torch.cuda.is_available():
                print(f"✅ CUDA - OK (Device: {torch.cuda.get_device_name(0)})")
            else:
                print("⚠️ CUDA - Not available (will use CPU)")
        except:
            print("❌ CUDA - Error checking")

# Install dependencies
dep_manager = DependencyManager()
dep_manager.install_packages()
dep_manager.verify_installation()

## 🧪 Step 3: Test Fixed Environment

In [ ]:
def test_fixed_environment():
    """Test that all the fixes work properly"""
    print("🧪 Testing Fixed Environment")
    print("=" * 40)
    
    # Test 1: GPUtil import
    print("\n1. Testing GPUtil import...")
    try:
        import GPUtil
        gpus = GPUtil.getGPUs()
        print(f"   ✅ GPUtil works! Found {len(gpus)} GPU(s)")
        if gpus:
            for i, gpu in enumerate(gpus):
                print(f"      GPU {i}: {gpu.name} ({gpu.memoryTotal}MB)")
    except ImportError:
        print("   ⚠️ GPUtil not available, testing alternatives...")
        
    # Test 2: PyTorch GPU detection
    print("\n2. Testing PyTorch GPU detection...")
    try:
        import torch
        if torch.cuda.is_available():
            print(f"   ✅ PyTorch CUDA available: {torch.cuda.get_device_name(0)}")
        else:
            print("   ⚠️ PyTorch CUDA not available - will use CPU")
    except Exception as e:
        print(f"   ❌ PyTorch test failed: {str(e)}")
    
    # Test 3: System info
    print("\n3. Testing system info...")
    try:
        import psutil
        memory = psutil.virtual_memory()
        print(f"   ✅ RAM: {memory.total / 1024**3:.1f}GB total")
        
        disk = psutil.disk_usage('/')
        print(f"   ✅ Disk: {disk.free / 1024**3:.1f}GB free")
    except Exception as e:
        print(f"   ❌ System info failed: {str(e)}")
    
    # Test 4: Required packages
    print("\n4. Testing required packages...")
    required_packages = ['torch', 'transformers', 'diffusers', 'opencv-python-headless', 'imageio']
    
    for package in required_packages:
        try:
            if package == 'opencv-python-headless':
                import cv2
                print(f"   ✅ {package} (cv2) - OK")
            else:
                __import__(package)
                print(f"   ✅ {package} - OK")
        except ImportError:
            print(f"   ❌ {package} - MISSING")
    
    print("\n✅ Environment test completed!")
    print("\n💡 You can now proceed with the rest of the notebook.")
    print("   If you encounter any issues, check the error messages above.")

# Run the test
test_fixed_environment()

## 🔄 Step 4: Continue with Original Notebook

The GPUtil issue has been fixed! You can now continue with the rest of your original notebook.

**What was fixed:**
1. ✅ GPUtil module is now properly installed
2. ✅ Alternative GPU detection methods added as fallbacks
3. ✅ Error handling improved for different environments
4. ✅ Dependencies updated and verified

**Next steps:**
- Continue with model downloads (Step 3 from original notebook)
- Set up the video generation pipeline (Step 4 from original notebook)
- Use the enhanced UI (Step 5 from original notebook)

The rest of your original notebook should now work without the GPUtil import errors!